In [1]:
import argparse
from logging import getLogger
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color
from recbole.data import create_dataset
from recbole.utils.case_study import full_sort_topk, full_sort_scores

import logging
from logging import getLogger

import torch
import pickle

from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color

import copy
from recbole.data.interaction import Interaction, cat_interactions

pd.set_option('display.max_rows', None)


In [2]:
# model_file = 'saved/SASRec-Nov-22-2021_08-08-52.pth'
# model_file = 'saved/SASRec-Dec-01-2021_15-15-09.pth'
model_file = 'saved/SASRec-Dec-05-2021_09-53-22.pth'

checkpoint = torch.load(model_file)
config = checkpoint['config']
# config.final_config_dict['load_col']['inter'].append('cityid')
config.final_config_dict['unused_col']['inter'] = ['exposure_time', 'exposure_time_list']
# config.final_config_dict['save_dataset'] = True
# config.final_config_dict['save_dataloaders'] = True

init_seed(config['seed'], config['reproducibility'])
init_logger(config)
logger = getLogger()
logger.info(config)

dataset = create_dataset(config)
logger.info(dataset)
train_data, valid_data, test_data = data_preparation(config, dataset)

init_seed(config['seed'], config['reproducibility'])
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
model.load_state_dict(checkpoint['state_dict'])
model.load_other_parameter(checkpoint.get('other_parameter'))

05 Dec 10:37    INFO  
General Hyper Parameters:
gpu_id = 1
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/pretrain
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 1}
eval_step = 1
stopping_step = 500
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = exposure_time


In [3]:
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
test_result = trainer.evaluate(test_data, load_best_model=False, show_progress=config['show_progress'])
print(test_result)

Evaluate   :   0%|                                                            | 0/5 [00:00<?, ?it/s]:   0%|                                   | 0/5 [00:00<?, ?it/s, GPU RAM: 2.38 G/79.35 G]:   0%|                                   | 0/5 [00:00<?, ?it/s, GPU RAM: 4.29 G/79.35 G]:   0%|                                   | 0/5 [00:00<?, ?it/s, GPU RAM: 4.29 G/79.35 G]:  60%|████████████████▏          | 3/5 [00:00<00:00, 26.11it/s, GPU RAM: 4.29 G/79.35 G]:  60%|████████████████▏          | 3/5 [00:00<00:00, 26.11it/s, GPU RAM: 4.29 G/79.35 G]:  60%|████████████████▏          | 3/5 [00:00<00:00, 26.11it/s, GPU RAM: 4.29 G/79.35 G]: 100%|███████████████████████████| 5/5 [00:00<00:00, 28.89it/s, GPU RAM: 4.29 G/79.35 G]


OrderedDict([('recall@10', 0.2001), ('mrr@10', 0.1271), ('ndcg@10', 0.1443), ('hit@10', 0.2001), ('precision@10', 0.02)])


In [4]:
# city_uid = test_data.dataset.inter_feat[test_data.uid_field][test_data.dataset.inter_feat['cityid'] == 1]
# city_data = copy.copy(test_data)
# city_data.dataset = copy.copy(test_data.dataset)
# if test_data.is_sequential:
#     _, index = (test_data.dataset.inter_feat[test_data.uid_field] == city_uid[:, None]).nonzero(as_tuple=True)
#     city_data.dataset.inter_feat = city_data.dataset.inter_feat[index]
# else:
#     city_data.uid_list = city_uid
#     city_data.user_df = dataset.join({test_data.uid_field: city_uid})
# city_result = trainer.evaluate(city_data, load_best_model=False, show_progress=config['show_progress'])
# print(city_result)

In [5]:
city_list = torch.unique(test_data.dataset.inter_feat['cityid'])
print(city_list)
city_res_dict = {
    'city_id': [],
    'total_inter_num': [],
    'train_inter_num': [],
    'valid_inter_num': [],
    'test_inter_num': [],
    'recall@10': [],
    'mrr@10': [],
    'ndcg@10': [],
    'hit@10': [],
    'precision@10': [],
}
for city_id in tqdm(city_list.numpy()):
    index = test_data.dataset.inter_feat['cityid'] == city_id
    city_uid = torch.unique(test_data.dataset.inter_feat[test_data.uid_field][index])
    city_data = copy.copy(test_data)
    city_data.dataset = copy.copy(test_data.dataset)
    if test_data.is_sequential:
        city_data.dataset.inter_feat = city_data.dataset.inter_feat[index]
    else:
        city_data.uid_list = city_uid
        city_data.user_df = dataset.join({test_data.uid_field: city_uid})
    city_result = trainer.evaluate(city_data, load_best_model=False, show_progress=False)
    city_res_dict['city_id'].append(city_id)
    city_res_dict['total_inter_num'].append((dataset.inter_feat['cityid'] == city_id).sum().item())
    city_res_dict['train_inter_num'].append((train_data.dataset.inter_feat['cityid'] == city_id).sum().item())
    city_res_dict['valid_inter_num'].append((valid_data.dataset.inter_feat['cityid'] == city_id).sum().item())
    city_res_dict['test_inter_num'].append(index.sum().item())
    city_res_dict['recall@10'].append(city_result['recall@10'])
    city_res_dict['mrr@10'].append(city_result['mrr@10'])
    city_res_dict['ndcg@10'].append(city_result['ndcg@10'])
    city_res_dict['hit@10'].append(city_result['hit@10'])
    city_res_dict['precision@10'].append(city_result['precision@10'])
    # print(city_id, city_result)
    # if city_id > 30:
    #     break

tensor([1, 2, 3])


100%|██████████| 3/3 [00:00<00:00,  6.75it/s]


In [6]:
city_res_dict['city_id'] = list(map(int, dataset.id2token('cityid', city_res_dict['city_id'])))
city_res = pd.DataFrame(city_res_dict)
city_res.head()

,city_id,total_inter_num,train_inter_num,valid_inter_num,test_inter_num,recall@10,mrr@10,ndcg@10,hit@10,precision@10
0,1,325625,294389,14352,16884,0.1976,0.1249,0.1420,0.1976,0.0198
1,1283,2033,1831,98,104,0.3365,0.2462,0.2673,0.3365,0.0337
2,361,1015,770,88,157,0.3758,0.2887,0.3096,0.3758,0.0376


In [7]:
city_res['total_inter_num'].sum()

764647

In [8]:
city_res['train_inter_num'].sum() + city_res['valid_inter_num'].sum() + city_res['test_inter_num'].sum()

764647

In [9]:
city_res.sort_values(by='recall@10', ascending=False).head(50)

,city_id,total_inter_num,train_inter_num,valid_inter_num,test_inter_num,recall@10,mrr@10,ndcg@10,hit@10,precision@10
2,1283,3405,3144,133,128,0.2969,0.2168,0.2360,0.2969,0.0297
0,1,718452,657863,28026,32563,0.1522,0.0887,0.1036,0.1522,0.0152
1,265,42790,40102,1254,1434,0.1297,0.0624,0.0781,0.1297,0.0130


In [32]:
city_df = pd.read_csv('~/city.txt', sep='\t')
print(len(city_df))
city_df = city_df[['mt_city_id', 'mt_city_name']]
city_df.columns = ['city_id', 'city_name']
city_df.head()

1226


,city_id,city_name
0,936,西华
1,842,盘州市
2,287,潮州
3,1195,福安
4,1203,东明县


In [33]:
print(len(city_res))

1178


In [34]:
new_city_res = pd.merge(city_res, city_df, how='left', on='city_id')
print(len(new_city_res))
new_city_res.head()

1178


,city_id,total_inter_num,train_inter_num,valid_inter_num,test_inter_num,recall@10,mrr@10,ndcg@10,hit@10,precision@10,city_name
0,482,3133,2929,99,105,0.1333,0.0554,0.0738,0.1333,0.0133,腾冲
1,324,15195,14355,411,429,0.0769,0.0232,0.0356,0.0769,0.0077,安顺
2,30,627762,593706,17075,16981,0.0775,0.0353,0.0451,0.0775,0.0077,深圳
3,73,248466,233518,7483,7465,0.0579,0.0283,0.0352,0.0579,0.0058,郑州
4,99,195176,185833,4672,4671,0.0612,0.0282,0.0359,0.0612,0.0061,南宁


In [37]:
new_city_res.sort_values(by='city_id', ignore_index=True, inplace=True)

In [38]:
new_city_res.head()

,city_id,total_inter_num,train_inter_num,valid_inter_num,test_inter_num,recall@10,mrr@10,ndcg@10,hit@10,precision@10,city_name
0,1,595922,555699,20158,20065,0.0778,0.0378,0.0471,0.0778,0.0078,北京
1,10,506161,476555,14815,14791,0.0721,0.0336,0.0425,0.0721,0.0072,上海
2,20,608105,574994,16567,16544,0.0800,0.0377,0.0476,0.0800,0.0080,广州
3,30,627762,593706,17075,16981,0.0775,0.0353,0.0451,0.0775,0.0077,深圳
4,40,204110,189901,7097,7112,0.0697,0.0338,0.0422,0.0697,0.0070,天津


In [39]:
new_city_res.to_csv('~/city_result.csv', sep='\t', index=False)